In [1]:
# from https://github.com/dpwe/calfonts/blob/add-bd/glcd-fontcreator-to-bdf.ipynb

# glcd-fontcreator-to-bdf.ipynb
#
# Converts the fontcreator2-format files used by openglcd
# into the X11 BDF format that can be ingested by u8g2.
#
# Dan Ellis dpwe@ 2022-02-20

In [2]:
from PIL import Image

In [43]:
# read fontcreator format file from openglcd

#foundry_name = "integratedmapping"
#font_name = "CalBlk36"
#font_weight = "bold"
#font_ptsize = 36
#font_descent = -8

#font_name = "CalLite24"
#font_weight = "light"
#font_ptsize = 24
#font_descent = -6

foundry_name = "Hitachi"
font_name = "System5x7"
font_weight = "normal"
font_ptsize = 7
font_descent = 0

parent_dir = ""
#parent_dir = "/Users/dpwe/Downloads/"

font_file = '{:s}{:s}.h'.format(parent_dir, font_name)

bytes = []

with open(font_file, 'rt') as f:
    in_brace = False
    in_comment = False
    line_has_comment = False
    last_comment_byte = 0
    for l in f:
        if '/*' in l:
            in_comment = True
        if '*/' in l:
            in_comment = False
        if in_comment:
            continue;
        if '{' in l:
            in_brace = True
            continue
        if '}' in l:
            in_brace = False
            continue
        if in_brace:
            # Strip trailing comment
            if '//' in l:
                l, comment = l.split('//')
                line_has_comment = True
            # Deal with comma-separated fields
            for field in l.split(','):
                field = field.strip()
                if field:
                    try:
                        val = int(field, base=16)
                        bytes.append(val)
                    except:
                        print("could not parse ", field)
            if line_has_comment:
                line_has_comment = False
                print("comment [bytes since last= ", len(bytes) - last_comment_byte, "]: ", comment.strip())
                last_comment_byte = len(bytes)

comment [bytes since last=  2 ]:  size of zero indicates fixed width font,
comment [bytes since last=  1 ]:  width
comment [bytes since last=  1 ]:  height
comment [bytes since last=  1 ]:  first char
comment [bytes since last=  1 ]:  char count
comment [bytes since last=  0 ]:  Fixed width; char width table not used !!!!
comment [bytes since last=  0 ]:  font data
comment [bytes since last=  5 ]:  (space)
comment [bytes since last=  5 ]:  !
comment [bytes since last=  5 ]:  "
comment [bytes since last=  5 ]:  #
comment [bytes since last=  5 ]:  $
comment [bytes since last=  5 ]:  %
comment [bytes since last=  5 ]:  &
comment [bytes since last=  5 ]:  '
comment [bytes since last=  5 ]:  (
comment [bytes since last=  5 ]:  )
comment [bytes since last=  5 ]:  *
comment [bytes since last=  5 ]:  +
comment [bytes since last=  5 ]:  ,
comment [bytes since last=  5 ]:  -
comment [bytes since last=  5 ]:  .
comment [bytes since last=  5 ]:  /
comment [bytes since last=  5 ]:  0
comment [bytes

In [44]:
# Interpreting the bytes
font_size = 256 * bytes[1] + bytes[0]
# Size of zero indicates fixed-width font
fixed_width = True if font_size == 0 else False
font_width = bytes[2]
font_height = bytes[3]
font_start_code = bytes[4]
font_num_codes = bytes[5]

if fixed_width:
  font_widths = [font_width] * font_num_codes
  font_width_entries = 0
else:
  font_width_entries = font_num_codes
  font_widths = bytes[6 : 6 + font_width_entries]

font_data = bytes[6 + font_width_entries:]

print("font_width=", font_width, "font_height=", font_height)

font_width= 5 font_height= 7


In [45]:
print(font_num_codes, font_widths[:10])
print(list(zip(range(32, 52), font_widths[:20])))

97 [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[(32, 5), (33, 5), (34, 5), (35, 5), (36, 5), (37, 5), (38, 5), (39, 5), (40, 5), (41, 5), (42, 5), (43, 5), (44, 5), (45, 5), (46, 5), (47, 5), (48, 5), (49, 5), (50, 5), (51, 5)]


In [46]:
# Bytes per glyph is 5 * width.  Height is 36, which won't fit in 4 bytes but will fit in 5.  
# So encoding is by column, with each column padded to fill bytes.

def bytes_to_glyph(bytes, height):
    bytes_per_col = (height + 7) // 8
    width = len(bytes) // bytes_per_col
    if width != (len(bytes) / bytes_per_col):  # Must be an integral number
        raise ValueError("got {:d} bytes but width is {:d} and bytes per col is {:d}".format(len(bytes), width, bytes_per_col))
    im = Image.new('1', (width, height), 0)
    # Actual encoding is in "stripes" of width bytes, providing 8 pixel rows at a time
    for row in range(bytes_per_col):
        for column in range(width):
            val = bytes[row * width + column]
            for pixel in range(min(8, height - 8 * row)):
                if val & (2 ** pixel) > 0:
                    im.putpixel((column, row * 8 + pixel), 1)
    return im

#im = bytes_to_glyph(font_data[40:80], 36)
print(sum(font_widths[:2]), sum(font_widths[:3]))
bytes_per_col = (font_height + 7) // 8
im = bytes_to_glyph(font_data[bytes_per_col * sum(font_widths[:3]) : bytes_per_col * sum(font_widths[:4])], font_height)

display(im)  # Displays as white against black.

10 15


In [47]:
# Convert all the characters into images
glyphs = {}   # Dict of ascii: image

bytes_per_col = (font_height + 7) // 8
data_pos = 0
for index in range(font_num_codes):
    code = font_start_code + index
    width = font_widths[index]
    bytes_this_glyph = bytes_per_col * width
    glyphs[code] = bytes_to_glyph(font_data[data_pos : data_pos + bytes_this_glyph], font_height)
    data_pos += bytes_this_glyph

In [48]:
display(glyphs[50])
print(glyphs[50].width)
print(font_width)
print(int(round(1000 * glyphs[50].width / font_width)))

5
5
1000


In [49]:
# And now, BDF export...

def bdf_encode_glyph(im, code, fontwidth, font_descent):
    result = []
    result.append('STARTCHAR U+{:04x}'.format(code))
    result.append('ENCODING {:d}'.format(code))
    result.append('SWIDTH {:d} 0'.format(int(round(1000 * im.width / fontwidth))))
    result.append('DWIDTH {:d} 0'.format(im.width + 1))   # openglcd glyphs <127 have implicit final blank column
    result.append('BBX {:d} {:d} 0 {:d}'.format(im.width, im.height, font_descent))
    result.append('BITMAP')
    bytes = (im.width + 7) // 8
    formatstr = '{{:0{:d}x}}'.format(2 * bytes)
    for row in range(im.height):
        val = 0
        for col in range(im.width):
            if im.getpixel((col, row)):
                val += (2 ** (8 * bytes - 1 - col))
        result.append(formatstr.format(val))
    result.append('ENDCHAR')
    return '\n'.join(result)

print(bdf_encode_glyph(glyphs[50], 50, font_width, font_descent))

STARTCHAR U+0032
ENCODING 50
SWIDTH 1000 0
DWIDTH 6 0
BBX 5 7 0 0
BITMAP
70
88
08
10
20
40
f8
ENDCHAR


In [50]:
# And the whole font

bdf_file = '{:s}{:s}.bdf'.format(parent_dir, font_name)

def write_bdf_file(filename, font_start_code, font_width, font_height, font_descent, glyphs):
    num_glyphs = len(glyphs)
    pointsize = glyphs[font_start_code].height
    with open(bdf_file, 'w') as f:
        f.write('STARTFONT 2.1\n')
        f.write('FONT -{:s}-{:s}-{:s}-r-normal--{:d}-{:d}-75-75-c-80-iso10646-1\n'.format(foundry_name, font_name, font_weight, font_ptsize, font_ptsize * 10))
        f.write('SIZE {:d} 75 75\n'.format(pointsize))
        f.write('FONTBOUNDINGBOX {:d} {:d} 0 {:d}\n'.format(font_height, font_width, font_descent))
        f.write('CHARS {:d}\n'.format(num_glyphs))
        for index in range(num_glyphs):
            code = font_start_code + index
            f.write(bdf_encode_glyph(glyphs[code], code, font_width, font_descent))
            f.write('\n')
        f.write('ENDFONT\n')

write_bdf_file(bdf_file, font_start_code, font_width, font_height, font_descent, glyphs)
print("Wrote", bdf_file)

Wrote System5x7.bdf


In [ ]:
# Then, to incorporate into U8g2:
#   $ cd github/u8g2/tools/bdfconv/
#   $ ./bdfconv -f 1 -m '32-127' -n System5x7 -o ~/Downloads/System5x7.c ~/Downloads/System5x7.bdf
# Then insert the content of the *.c output file into
#   ~/Documents/Arduino/libraries/U8g2/src/clib/u8g2_fonts.c
# and its signature into
#   ~/Documents/Arduino/libraries/U8g2/src/clib/u8g2.h
# then your new font should be available.